# Lets have genie spaces as production assets

Genie is a fantastic tool, and with the Genie Research Agent, it becomes even more powerful! Let’s explore the exciting journey of migrating Genie from the UI to robust production assets while developing a seamless CI/CD strategy to optimize our Genie spaces. 

- Check the documentation about create/select Genie Spaces with the API [here](https://docs.databricks.com/aws/en/genie/conversation-api#create-or-select-a-genie-space)
- More about Research Agent [here](https://docs.databricks.com/aws/en/genie/research-agent)
### Lets import a genie space and review it on a readable format

First, we'll gather the right info about the Workspace and space ID. Then, using Databricks secrets securely, we'll manage our scope and secrets to retrieve the JSON that outlines the genie space setup in a clean format. It's going to be a great step forward!

- Check for the specifics for how to use the API to GET a genie space [here](https://docs.databricks.com/api/workspace/genie/getspace)
- Handle Secrects securly in Databricks [here](https://docs.databricks.com/aws/en/security/secrets/)

In [0]:
import requests
import json

#Here put the account information 
databricks_workspace = "<your-databricks-workspace>"  
space_id = "<space_id>" 
token = dbutils.secrets.get(scope="<Your_Scope>", key="<Your_Key>")  # Verify the secret scope and key are correct and the token has necessary permissions. Leverage Databricks Secrects. 

# This is the form of a get request for genie Space Information 
url = f"{databricks_workspace}/api/2.0/genie/spaces/{space_id}?include_serialized_space=true" 
# Note: ?include_serialized_space=true for including all parameters
headers = {"Authorization": f"Bearer {token}"}
response = requests.get(url, headers=headers)
response.raise_for_status()

response_json = response.json() #Let's keep one version to be used later 
response_json_readable = response.json() #Lets transform the json to be human readable

# Lets see a nicer format for the human eye 
serialized = response_json_readable.get("serialized_space")
if isinstance(serialized, str):
    try:
        response_json_readable["serialized_space"] = json.loads(serialized)
    except json.JSONDecodeError:
        # API lied to you, apparently. Leave it as-is.
        pass

print(json.dumps(response_json_readable, indent=2))

%md
### Now, let's import and export a Genie Space

Now that we understand how to get the Genie Space, let's use that same response to create a new Genie Space with that information in a specific title and in the desired location. Using the **response_json** from the first code block we can create a new genie space. *_It's necessary to run the first code block in the same session._

- Check for the specifics for how to use the API to POST (Create) a genie space [here](https://docs.databricks.com/api/workspace/genie/createspace)

In [0]:
import requests
import json
from databricks.sdk import WorkspaceClient

# Define the correct workspace and url on the desire location to create the new Genie Space
databricks_workspace = "<your-databricks-workspace>"  
url = f"{databricks_workspace}/api/2.0/genie/spaces" 

# In this case lets mark it by user name (This is optional)
user_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()

space_title = 'API-created Genie Space by ' + user_name.split('@')[0]

token = dbutils.secrets.get(scope="<Your_Scope>", key="<Your_Key>")  # Verify the secret scope and key are correct and the token has necessary permissions.

#Provide to Authorization details
headers = {
    "Authorization": f"Bearer {token}"
}

#Use the information 
payload = {"serialized_space": response_json['serialized_space'], "warehouse_id": response_json['warehouse_id'], 'title': space_title}

create_response = requests.post(url, headers=headers, params=payload)
create_response_json = create_response.json()
create_response_json

### Finally lets Modify an existing genie space

For this final part, here is an example of how to modify a genie space, in this case the description


- Check for the specifics for how to use the API to PATCH (Create) a genie space [here](https://docs.databricks.com/api/workspace/genie/updatespace)
- Yes, you can also delete genie spaces from the API, check [here](https://docs.databricks.com/api/workspace/genie/trashspace)

In [0]:
# Genie API endpoint for space update
databricks_workspace = "<your-databricks-workspace>"  
space_id = "<space_id>"  #Genie Space ID which its intended to update
token = dbutils.secrets.get(scope="<Your_Scope>", key="<Your_Key>") # Verify the secret scope and key are correct and the token has necessary permissions.

#Lets update the description of the Genie Space
space_description = "This dataset represents the information of a call center located in the us. It contains transactional data of the historical calls and metrics on the service agents"

# This is the form of a get request for genie Space Information 
url = f"{databricks_workspace}/api/2.0/genie/spaces/{space_id}" 
headers = {"Authorization": f"Bearer {token}"}

payload = {"description": space_description}

update_response = requests.patch(url, headers=headers, params=payload)
update_response_json = update_response.json()
update_response_json


In [0]:
#Same process for the credentials  
databricks_workspace = "<your-databricks-workspace>"  
space_id = "<space_id>"  #Genie Space ID which its intended to delete
token = dbutils.secrets.get(scope="<Your_Scope>", key="<Your_Key>") 

url = f"{databricks_workspace}/api/2.0/genie/spaces/{space_id}" 
headers = {"Authorization": f"Bearer {token}"}


delete_response = requests.delete(url, headers=headers)
delete_response_json = delete_response.json()
update_response_json